In [83]:
import pandas as pd
import re
import numpy as np

AADT = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRXk2ssI2wP32cbxd7gJecku7nS9Mjim7Ed3dorQsgalcBYC6KbxpWKlx0ClBsmsgAcbf5QzQqt2tsy/pub?output=csv")

In [84]:
def clean_crash_summary(file_path):
    # Read the CSV file
    crash_summary = pd.read_csv(file_path)

    # Filter and select relevant columns
    crash_summary_clean = crash_summary[(crash_summary['Jurisdiction'] == 'State Road') &
                                        (~crash_summary['Weather Condition'].isna())].drop(columns=['Collision Type'])

    # Create binary columns and convert categorical columns to category type
    crash_summary_clean = crash_summary_clean.assign(
        **{
            'School Zone': crash_summary_clean['School Zone'].apply(lambda x: 1 if x == 'Y' else 0),
            'Intersection Related': crash_summary_clean['Intersection Related'].apply(lambda x: 1 if x == 'Y' else 0),
            'Damage Threshold Met': crash_summary_clean['Damage Threshold Met'].apply(lambda x: 1 if x == 'Y' else 0),
            'Hit and Run': crash_summary_clean['Hit and Run'].apply(lambda x: 1 if x == 'Y' else 0),
            'Passengers Involved': crash_summary_clean['Passengers Involved'].apply(lambda x: 1 if x == 'Y' else 0),
            'Commercial Carrier Involved': crash_summary_clean['Commercial Carrier Involved'].apply(lambda x: 1 if x == 'Y' else 0),
            'School Bus Involved': crash_summary_clean['School Bus Involved'].apply(lambda x: 1 if x == 'Y' else 0),
            'Agency': crash_summary_clean['Agency'].astype('category'),
            'Weather Condition': crash_summary_clean['Weather Condition'].astype('category'),
            'Lighting Condition': crash_summary_clean['Lighting Condition'].astype('category'),
            'Injury Severity': crash_summary_clean['Injury Severity'].astype('category')
        }
    )
    
    return crash_summary_clean

In [85]:
Car_Crash = clean_crash_summary("https://docs.google.com/spreadsheets/d/e/2PACX-1vRJhryMDLGWP2PxsaXiDYb5PdBN_vmZxV0aieOFUJNuD5OBBJTR927qUVRnPFBg_5iDbFgxWzDWPvC9/pub?output=csv")
Car_Crash.reindex(np.arange(len(Car_Crash)))

,Collision Report Number,Collision Date,County,City,Jurisdiction,Agency,Primary Trafficway,Secondary Trafficway,Block Number,Mile Post,...,Injury Severity,Object Struck,Damage Threshold Met,Hit and Run,Motor Vehicles Involved,Passengers Involved,Commercial Carrier Involved,School Bus Involved,Pedestrians Involved,Pedalcyclists Involved
0,EC12134,1/1/2022 12:13:00 AM,King,Bellevue,State Road,State Patrol,WB I 90,FROM SB I-405,NaN,9.72,...,No Injury Collision,barrier,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,EC09140,1/1/2022 12:26:00 AM,Kitsap,Bremerton,State Road,State Patrol,SR 3,NaN,NaN,36.00,...,No Injury Collision,Concrete/Jersey Barrier,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,GARFIELD RD,11900,NaN,...,No Injury Collision,NaN,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
3,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,GARFIELD RD,11900,NaN,...,No Injury Collision,NaN,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
4,EC09574,1/1/2022 12:48:00 AM,Grant,Mattawa,State Road,State Patrol,SR 243,MORRISON ROAD,NaN,18.00,...,No Injury Collision,none,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44601,ED20862,12/30/2022 11:25:00 PM,Clark,Vancouver,State Road,State Patrol,I-205,PADDEN PARKWAY,NaN,34.85,...,No Injury Collision,NaN,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0
44602,ED21608,12/30/2022 11:33:00 PM,Cowlitz,Woodland,State Road,State Patrol,INTERSTATE 5,DIKE ROAD,NaN,22.80,...,No Injury Collision,Guardrail,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
44603,ED20667,12/30/2022 11:50:00 PM,King,North Bend,State Road,State Patrol,WB I-90,MP 53,NaN,53.00,...,No Injury Collision,Over Embankment,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
44604,ED21604,12/30/2022 11:51:00 PM,King,Federal Way,State Road,State Patrol,SOUTHBOUND I-5,W/B STATE ROUTE 18,NaN,143.00,...,No Injury Collision,NaN,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0


In [86]:
validSR = AADT['StateRouteNumber'].unique().tolist()
Trafficway = ["Primary Trafficway","Secondary Trafficway"]
dict = {
  0: [],
  1: [],
}

In [87]:
for i in np.arange(2):

    for x in Car_Crash[Trafficway[i]]:
        #Gets the state roads numbers from the primary trafficway. Has to match "[String of nondigit text]integer" 
        #Does remove some values that don't follow road convenient of state roads. Does account for roads like 123th Highway
        if type(x) == str:
            if (not re.match("\D+\d+", x) == None):
                if int(re.findall(r'\d+', x)[0]) in validSR:
                    dict[i].append(int(re.findall(r'\d+', x)[0]))
                else:
                    dict[i].append(None)
            else:
                dict[i].append(None)
        else:
            dict[i].append(None)

State_Road_Num = []
for x in np.arange(len(dict[0])):
    #From the 2 lists of state road numbers, it will first the state road number of the primary trafficway
    #If that is not available, it takes the state road number of the secondary trafficway
    if not dict[0][x] == None:
        State_Road_Num.append(dict[0][x])
    else:
        State_Road_Num.append(dict[1][x])

In [88]:
State_Road_Num.count(None) / float(len(State_Road_Num))

0.02654351432542707

In [89]:

Car_Crash.insert(7,"Primary Road Number", dict[0])
Car_Crash.insert(9,"Secondary Road Number", dict[1])
Car_Crash.insert(10, "Associated State Road Number", State_Road_Num)
Car_Crash.head()[["Primary Road Number","Secondary Road Number","Associated State Road Number"]]

,Primary Road Number,Secondary Road Number,Associated State Road Number
0,90.0,405.0,90.0
1,3.0,NaN,3.0
2,2.0,NaN,2.0
3,2.0,NaN,2.0
4,243.0,NaN,243.0


In [92]:
mileposts = []
for x in AADT["Location"]:
    match = re.search("(MILEPOST) (\d+.\d+)",x)
    mileposts.append(float(match.group(2)))

AADT.insert(6, "Mile Posts", mileposts)

In [95]:
len(Car_Crash['Associated State Road Number'])

44606

In [96]:
Crash_AADT = []
for x in np.arange(len(Car_Crash)):
    if not np.isnan(Car_Crash['Associated State Road Number'].iloc[x]):
        if Car_Crash['Mile Post'].iloc[x] > -1:
            SR = AADT[['Mile Posts', 'AADT']].loc[AADT['StateRouteNumber'] == Car_Crash['Associated State Road Number'].iloc[x]]
            SR.index = np.arange(len(SR))
            # Calculate absolute differences
            abs_diff = np.abs(SR['Mile Posts'] - Car_Crash['Mile Post'].iloc[x])
            
            # Find the index of the minimum absolute difference
            min_index = np.argmin(abs_diff)

            Crash_AADT.append(SR['AADT'].iloc[min_index])
        else:
            R = AADT['AADT'].loc[AADT['StateRouteNumber'] == int(Car_Crash['Associated State Road Number'].iloc[0])]
            Crash_AADT.append(R.median())
    else:
        Crash_AADT.append(None)

In [97]:
Car_Crash.insert(Car_Crash.shape[1], "AADT", Crash_AADT)

In [98]:
Car_Crash.head()

,Collision Report Number,Collision Date,County,City,Jurisdiction,Agency,Primary Trafficway,Primary Road Number,Secondary Trafficway,Secondary Road Number,...,Object Struck,Damage Threshold Met,Hit and Run,Motor Vehicles Involved,Passengers Involved,Commercial Carrier Involved,School Bus Involved,Pedestrians Involved,Pedalcyclists Involved,AADT
0,EC12134,1/1/2022 12:13:00 AM,King,Bellevue,State Road,State Patrol,WB I 90,90.0,FROM SB I-405,405.0,...,barrier,1,0,1,0,0,0,0,0,84000.0
1,EC09140,1/1/2022 12:26:00 AM,Kitsap,Bremerton,State Road,State Patrol,SR 3,3.0,NaN,NaN,...,Concrete/Jersey Barrier,1,0,1,0,0,0,0,0,80000.0
2,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,2.0,GARFIELD RD,NaN,...,NaN,0,0,2,1,0,0,0,0,5400.0
3,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,2.0,GARFIELD RD,NaN,...,NaN,1,0,2,1,0,0,0,0,5400.0
4,EC09574,1/1/2022 12:48:00 AM,Grant,Mattawa,State Road,State Patrol,SR 243,243.0,MORRISON ROAD,NaN,...,none,1,1,2,1,0,0,0,0,5200.0


In [99]:
Car_Crash.iloc[3594]

Collision Report Number                      EC21598
Collision Date                  2/4/2022 12:05:00 PM
County                                         Clark
City                                       Vancouver
Jurisdiction                              State Road
Agency                                  State Patrol
Primary Trafficway                               503
Primary Road Number                              NaN
Secondary Trafficway            NE 65T ST           
Secondary Road Number                            NaN
Associated State Road Number                     NaN
Block Number                                     NaN
Mile Post                                        0.2
School Zone                                        0
Intersection Related                               1
Weather Condition                           Overcast
Lighting Condition                          Daylight
Injury Severity                  No Injury Collision
Object Struck                                 

In [100]:
Car_Crash["AADT"].isna().sum()

1184

In [101]:
float(len(Car_Crash["AADT"]))

44606.0

In [103]:
(float(len(Car_Crash["AADT"])) - Car_Crash["AADT"].isna().sum())/float(len(Car_Crash["AADT"]))

0.9734564856745729

In [104]:
Car_Crash.dropna(subset=['AADT'], inplace=True)

In [105]:
Car_Crash.to_csv('Car_Crash_Cleaned_AADT.csv', index=False)